# 不同的Memory工具
- 短时记忆： 存储在内存中
- 构建记忆实体清单
- 接入知识图谱
- 长对话在内存中的处理方式
- 长时记忆的实现方式

### 1 短时记忆 内存中的短时记忆
- ConversationBufferMemory
- ConversationBufferWindowMemory

In [16]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()
memory.chat_memory.add_user_message("你好，我是人类")
memory.chat_memory.add_ai_message("你好，我是人工智能助手 凯影，有什么需要帮助的吗")

memory.load_memory_variables({})

{'history': 'Human: 你好，我是人类\nAI: 你好，我是人工智能助手 凯影，有什么需要帮助的吗'}

In [17]:
# 实现一个最近的对话窗口，超过窗口条数的对话将被删除
from langchain.memory import ConversationBufferWindowMemory
memory = ConversationBufferWindowMemory(k=1)
# 对话条目
memory.save_context({'input': "你好，我是人类"}, {"output":"你好，我是人工智能助手XXX，有什么需要帮助的吗"})
memory.save_context({'input': "你好，我饿了"}, {"output":"你有什么忌口吗？我给你提供食谱"})
# 测试输出
memory.load_memory_variables({})

{'history': 'Human: 你好，我饿了\nAI: 你有什么忌口吗？我给你提供食谱'}

### 2 构建记忆实体清单 
- ConversationEntityMemory

In [39]:
from langchain_ollama import OllamaLLM
from langchain.memory import ConversationEntityMemory

# llm = OllamaLLM(model="deepseek-r1:1.5b", temperature= 0)
llm = OllamaLLM(model="qwen2.5:latest", temperature= 0)

memory = ConversationEntityMemory(llm=llm)
_input = {
  "input":"胡万和麻匪在半夜火拼，九桶和四桶被打死了，胡万被埋了"
}

memory.load_memory_variables(_input)

{'history': '', 'entities': {'胡万': '', '麻匪': '', '九桶': '', '四桶': ''}}

In [40]:
memory.save_context(_input,
  {
    "output":"好刺激，我也想加入"
  }
)
memory.load_memory_variables({"input":"我该加入谁？"})

{'history': 'Human: 胡万和麻匪在半夜火拼，九桶和四桶被打死了，胡万被埋了\nAI: 好刺激，我也想加入', 'entities': {}}

### 3 使用知识图谱构建记忆 
- ConversationKGMemory

In [35]:
from langchain_ollama import OllamaLLM
from langchain.memory import ConversationKGMemory

# llm = OllamaLLM(model="deepseek-r1:1.5b")
llm = OllamaLLM(model="qwen2.5:latest")

memory = ConversationKGMemory(llm=llm)

# 构建知识谱
memory.save_context(
  {"input":"帮我找一下发财"},
  {"output":"请问发财是什么？"}
)
memory.save_context(
  {"input":"发财是一只黑色小猫咪"},
  {"output":"好的我知道了。"}
)

In [36]:
# 如何使用知识谱记忆
# 增加一条条目
memory.load_memory_variables({"input":"发财是什么？"})

{'history': 'On 发财: 发财 是 黑色小猫咪.'}

In [37]:
# 如何使用知识谱记忆
memory.get_current_entities("发财喜欢做什么？")

['发财']

In [38]:
# 获取知识图谱 三元结构拆解
memory.get_knowledge_triplets("发财喜欢到处跑")

[KnowledgeTriple(subject='D衰退', predicate='likes to', object_='run everywhere')]

### 4 长对话在内存中的处理方式 ConversationSummaryMemory
- 总结摘要
   - ConversationSummaryMemory
   - ChatMessageHistory 
- 计算Token
   - ConversationTokenBufferMemory

In [43]:
from langchain.memory import ConversationSummaryMemory
from langchain_ollama import OllamaLLM

llm = OllamaLLM(model="llama3.1:8b",temperature=0)

memory = ConversationSummaryMemory(llm=llm)

memory.save_context(
  {"input":"帮我找一下发财"},
  {"output":"请问发财是什么？"}
)
memory.save_context(
  {"input":"发财是一只黑色小猫咪"},
  {"output":"好的我知道了。"}
)

In [ ]:
# 把上面的对话内容摘要成一句话
memory.load_memory_variables({})

{'history': 'The human asks the AI to find something called "发财", but the AI is initially unsure. The human clarifies that "发财" refers to a black cat, and the AI understands what it means now.'}

In [46]:
messages = memory.chat_memory.messages
memory.predict_new_summary(messages,"")

'The human asks the AI to find something called "发财" (which translates to "getting rich"). The AI is unsure what "发财" refers to, but the human clarifies that it\'s a black cat. The AI acknowledges understanding now.'

In [48]:
# 通过ChatMessageHistory快速获得对话摘要

from langchain.memory import ChatMessageHistory

history = ChatMessageHistory()
history.add_user_message("你好，我是人类")
history.add_ai_message("你好，我是发财，一只快乐的小猫咪")

memory = ConversationSummaryMemory.from_messages(
  llm=OllamaLLM(model="llama3.1:8b",temperature=0),
  chat_memory=history,
  return_messages=True
)
memory.buffer

'Here\'s the updated summary:\n\nThe human greets the AI in Chinese, saying "你好，我是人类" (nǐ hǎo, wǒ shì rén lèi). The AI responds in a playful tone, introducing itself as "发财，一只快乐的小猫咪" (fā cái, yī zhī kuài lè de xiǎo māo mī), which roughly translates to "Get rich, a happy little cat".'

- 当对话持续进行且对话内容很多的时候
- 我们可以使用ConversationSummaryBufferMemory来存对话摘要
- 这是一种非常有用的方式，他会根据Token消耗数目来自动判断是否需要进行摘要
- 比较久的对话会被删除， 在删除前会进行摘要

In [ ]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain_ollama import OllamaLLM

llm = OllamaLLM(model="llama3.1:8b",temperature=0)

# 可以根据Token 的长度让llm自行总结前面的对话内容
memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=100,return_messages=True)
# memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=20,return_messages=True)

memory.save_context(
  {"input":"帮我找一下发财"},
  {"output":"请问发财是什么？"}
)
memory.save_context(
  {"input":"发财是一只黑色小猫咪"},
  {"output":"好的我知道了。"}
)
memory.save_context(
  {"input":"发财喜欢在房间里到处跑"},
  {"output":"好的我知道的， 还有别的特点吗？。"}
)

memory.buffer

[SystemMessage(content='Here is the new summary:\n\nThe human asks the AI to find something called "发财", which the AI doesn\'t understand. The human explains that "发财" refers to a black cat.\n\nLet me know when you\'re ready to add more lines of conversation!', additional_kwargs={}, response_metadata={}),
 AIMessage(content='好的我知道了。', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='发财喜欢在房间里到处跑', additional_kwargs={}, response_metadata={}),
 AIMessage(content='好的我知道的， 还有别的特点吗？。', additional_kwargs={}, response_metadata={})]

In [60]:
memory.load_memory_variables({})

{'history': [SystemMessage(content='Here is the new summary:\n\nThe human asks the AI to find something called "发财", which the AI doesn\'t understand. The human explains that "发财" refers to a black cat.\n\nLet me know when you\'re ready to add more lines of conversation!', additional_kwargs={}, response_metadata={}),
  AIMessage(content='好的我知道了。', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='发财喜欢在房间里到处跑', additional_kwargs={}, response_metadata={}),
  AIMessage(content='好的我知道的， 还有别的特点吗？。', additional_kwargs={}, response_metadata={})]}

#### ConversationTokenBuffer 使用Token长度来决定什么时候刷新内存

In [64]:
from langchain.memory import ConversationTokenBufferMemory
from langchain_ollama import OllamaLLM

llm = OllamaLLM(model="llama3.1:8b",temperature=0)

memory = ConversationTokenBufferMemory(llm=llm, max_token_limit=200,return_messages=True)
# memory = ConversationTokenBufferMemory(llm=llm, max_token_limit=20,return_messages=True)

memory.save_context(
  {"input":"帮我找一下发财"},
  {"output":"请问发财是什么？"}
)
memory.save_context(
  {"input":"发财是一只黑色小猫咪"},
  {"output":"好的我知道了。"}
)
memory.save_context(
  {"input":"发财喜欢在房间里到处跑"},
  {"output":"好的我知道的， 还有别的特点吗？。"}
)
memory.save_context(
  {"input":"发财喜欢睡觉，发财喜欢拉粑粑"},
  {"output":"好的我知道的。还有别的吗？"}
)
memory.save_context(
  {"input":"发财喜欢钻到犄角旮旯，就这些了"},
  {"output":"好的我知道的"}
)
memory.buffer

[HumanMessage(content='发财喜欢在房间里到处跑', additional_kwargs={}, response_metadata={}),
 AIMessage(content='好的我知道的， 还有别的特点吗？。', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='发财喜欢睡觉，发财喜欢拉粑粑', additional_kwargs={}, response_metadata={}),
 AIMessage(content='好的我知道的。还有别的吗？', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='发财喜欢钻到犄角旮旯，就这些了', additional_kwargs={}, response_metadata={}),
 AIMessage(content='好的我知道的', additional_kwargs={}, response_metadata={})]

### 5 长时记忆的实现方式

通过向量数据库存之前的对话内容，有的向量数据库会自动摘要，每次对话的时候，都会从向量数据库里查询文档或者对话历史

In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.memory import ConversationBufferMemory
from langchain.vectorstores import FAISS
from langchain_ollama import OllamaEmbeddings, OllamaLLM

In [76]:
memory = ConversationBufferMemory()
memory.save_context(
  {"input":"帮我找一下发财"},
  {"output":"请问发财是什么？"}
)
memory.save_context(
  {"input":"发财是一只黑色小猫咪"},
  {"output":"好的我知道了。"}
)
memory.save_context(
  {"input":"发财喜欢在房间里到处跑"},
  {"output":"好的我知道的， 还有别的特点吗？。"}
)
memory.save_context(
  {"input":"发财喜欢睡觉，发财喜欢拉粑粑"},
  {"output":"好的我知道的。还有别的吗？"}
)
memory.save_context(
  {"input":"发财喜欢钻到犄角旮旯，就这些了"},
  {"output":"好的我知道的"}
)
memory.buffer

'Human: 帮我找一下发财\nAI: 请问发财是什么？\nHuman: 发财是一只黑色小猫咪\nAI: 好的我知道了。\nHuman: 发财喜欢在房间里到处跑\nAI: 好的我知道的， 还有别的特点吗？。\nHuman: 发财喜欢睡觉，发财喜欢拉粑粑\nAI: 好的我知道的。还有别的吗？\nHuman: 发财喜欢钻到犄角旮旯，就这些了\nAI: 好的我知道的'

In [83]:
# 定义数据库
# Define the vectorstore
bed = OllamaEmbeddings(model="nomic-embed-text:latest")
texts = f"{memory.buffer}".split('\n')
vectorstore = FAISS.from_texts(
  texts=texts,
  embedding=bed
)
FAISS.save_local(vectorstore, "test_faiss")

In [88]:
# 验证数据看

FAISS.load_local("test_faiss", bed,
                 allow_dangerous_deserialization=True).similarity_search("发财")

[Document(id='32d192c6-092c-4eec-8876-5363cfb7e808', metadata={}, page_content='AI: 请问发财是什么？'),
 Document(id='5f5ae85d-730b-4a0b-a2d8-43c6c26bb6ed', metadata={}, page_content='Human: 发财是一只黑色小猫咪'),
 Document(id='5b2b6d77-cf65-4a31-8461-b7e1b58b82f9', metadata={}, page_content='AI: 好的我知道了。'),
 Document(id='3f291f1d-d5a2-4536-949c-469b476d4938', metadata={}, page_content='Human: 帮我找一下发财')]

In [ ]:
from langchain.vectorstores import FAISS
from langchain_ollama import OllamaEmbeddings
from langchain.memory import VectorStoreRetrieverMemory

r1 = FAISS.load_local("test_faiss", bed, allow_dangerous_deserialization=True)
r2 = r1.as_retriever(search_kwargs={"k":4})

memory2 = VectorStoreRetrieverMemory(retriever=r2)
memory2.load_memory_variables({"prompt":"发财是什么？"})

## memory2 是新建的memory 从 r1数据库中来

{'history': 'AI: 请问发财是什么？\nAI: 好的我知道的。还有别的吗？\nAI: 好的我知道的， 还有别的特点吗？。\nHuman: 发财是一只黑色小猫咪'}